In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Embedding size
m = 16

# Hidden size
hidden_size = 128

# Stack initialized?
init = False

# RNN vars
max_length  = 100
vocab_size  = 2          # Input vocabulary
seq_length  = 20
batch_size  = 32

In [3]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [4]:
s = None
V = None

In [5]:
# Pop operation helpers
def check(stack, strengths, remaining, read, idx):
    # Bottom of stack
    return idx >= 0 #tf.logical_and(idx >= 1, remaining != 0)

def update(stack, strengths, remaining, read, idx):
    # Amount we can use at this step
    this_qty = tf.minimum(remaining, strengths[:,:,idx:idx+1])

    # Update read value
    old_read = read
    read = tf.reshape(read + tf.reshape(this_qty * V[:,:,idx:idx+1], tf.shape(read)), tf.shape(read))  # for shape constraints

    # Update remaining strength
    remaining = tf.reshape(remaining - this_qty, tf.shape(remaining))

    # Update strengths
    before = strengths[:,:,:idx]
    this   = tf.reshape(tf.sub(strengths[:,:,idx:idx+1], this_qty), (-1, 1, 1))
    after  = strengths[:,:,idx+1:]

    strengths = tf.reshape(tf.concat(2, [before, this, after]), tf.shape(strengths), name="strength_cat")

    # Update index
    idx = idx - 1

    return (stack, strengths, remaining, read, idx)

In [6]:
def stack_update(d, u, v):
    '''
    Performs an update to the neural stack.
    
    Args:
      d: Push probability.
      u: Pop probability.
      v: Push value.
    
    Returns:
      r: The value read from the stack.
    '''
    global s, V, m, init
    
    # Change shapes
    d = tf.expand_dims(d, -1)
    u = tf.expand_dims(u, -1)
    v = tf.expand_dims(v, -1)
    
    # Infer batch size
    batch_size = tf.shape(d)[0]
    
    if init:
        # Infer stack size
        stack_size = tf.shape(V)[2]
        
        # Perform initializations
        read0     = tf.zeros((batch_size, m))      # Read value
        idx0      = stack_size - 1                  # Index into the stack
        
        initialization = (V, s, u, read0, idx0)
        pop_operation = tf.while_loop(check, update, initialization)
        
        # Update strengths and perform read
        s = pop_operation[1]
        r = pop_operation[3]
        
        # Perform push
        V = tf.concat(2, [V, v])
        s = tf.concat(2, [s, d])
        
    else:
        r = tf.zeros((batch_size, m), dtype=np.float32)
        init = True
        
        # Initialize stack
        V = v
        s = d
    
    return r

In [7]:
input = [tf.placeholder(tf.float32, shape=(batch_size, vocab_size), name="inp%i" % t) for t in range(seq_length)]
label = tf.placeholder(tf.float32, shape=(batch_size, 21), name="label")

In [8]:
cell = tf.nn.rnn_cell.GRUCell(hidden_size)
state = cell.zero_state(batch_size, tf.float32)
outputs = []

In [9]:
# For getting the push probability
W_d = tf.Variable(tf.random_normal((hidden_size, 1), stddev=0.01))
b_d = tf.Variable(np.zeros((1,)), dtype=tf.float32)

# For getting the pop probability
W_u = tf.Variable(tf.random_normal((hidden_size, 1), stddev=0.01))
b_u = tf.Variable(np.zeros((1,)), dtype=tf.float32)

# For getting the value to be pushed
W_v = tf.Variable(tf.random_normal((hidden_size, m), stddev=0.01))
b_v = tf.Variable(np.zeros((m,)), dtype=tf.float32)

In [10]:
# Initialize first read to all 0's
r_t = tf.zeros((batch_size, m))

with tf.variable_scope('rnn_unfolding') as varscope:
    for input_ in input:
        combined_input = tf.concat(1, [input_, r_t])
        output, state  = cell(combined_input, state)
        
        # Calculate d, u, v
        d_t = tf.sigmoid(tf.matmul(output, W_d) + b_d)
        u_t = tf.sigmoid(tf.matmul(output, W_u) + b_u)
        v_t = tf.tanh(tf.matmul(output, W_v) + b_v)
        
        # Perform stack operation
        r_t = stack_update(d_t, u_t, v_t)
        
        outputs.append(output)
        varscope.reuse_variables()

In [11]:
# Attach MLP to last output
W = tf.Variable(tf.random_normal((128, 21), stddev=0.01))
b = tf.Variable(np.zeros((21,)), dtype=tf.float32)

output = tf.sigmoid(tf.matmul(outputs[-1], W) + b)

In [12]:
# Loss
clipped_output         = tf.clip_by_value(output, 1e-10, 1.0)
clipped_1_minus_output = tf.clip_by_value(1 - output, 1e-10, 1.0)

loss = -tf.reduce_sum(label * tf.log(clipped_output) + (1 - label) * tf.log(clipped_1_minus_output))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output, 1), tf.argmax(label, 1)), tf.float32))

In [13]:
# Optimizer
train_op = tf.train.AdamOptimizer().minimize(loss)

In [14]:
# Initialize
sess.run(tf.initialize_all_variables())

In [15]:
def one_hotify(vector, output_vocabulary_size):
    # Create the result vector
    vector_one_hot = np.zeros(list(vector.shape) + [output_vocabulary_size])
    
    # Use fancy indexing to activate positions
    vector_one_hot[list(np.indices(vector.shape)) + [vector]] = 1
    
    return vector_one_hot

In [16]:
# Train
losses     = []
accuracies = []

for iter_ in range(2 * 10 ** 4):
    # Generate samples randomly
    x_   = np.random.randint(0, 2, (batch_size, seq_length))
    x_1h = one_hotify(x_, vocab_size)

    y_   = np.sum(x_, 1)
    y_1h = one_hotify(y_, seq_length + 1)   # because #_output_symbols = seq_length + 1
    
    # Calculate loss and backprop
    feed_dict = {input[t]: x_1h[:,t,:] for t in range(seq_length)}
    feed_dict.update({label: y_1h})
    loss_t, acc_t, _ = sess.run((loss, accuracy, train_op), feed_dict)
    
    losses.append(loss_t)
    accuracies.append(acc_t * 100)
    
    if iter_ % (10 ** 2) == 0 and iter_ > 0:
        print 'Iteration: %5d Loss: %3.4f Accuracy: %3.2f%%' % (iter_, np.mean(losses), np.mean(accuracies))

Iteration:   100 Loss: 149.7176 Accuracy: 12.13%
Iteration:   200 Loss: 125.7907 Accuracy: 14.75%
Iteration:   300 Loss: 117.5809 Accuracy: 15.44%
Iteration:   400 Loss: 113.7701 Accuracy: 15.79%
Iteration:   500 Loss: 111.3110 Accuracy: 15.66%
Iteration:   600 Loss: 109.5223 Accuracy: 15.97%
Iteration:   700 Loss: 107.8388 Accuracy: 16.38%
Iteration:   800 Loss: 106.4812 Accuracy: 16.80%
Iteration:   900 Loss: 104.9568 Accuracy: 17.70%
Iteration:  1000 Loss: 102.9972 Accuracy: 19.36%
Iteration:  1100 Loss: 100.8224 Accuracy: 21.54%
Iteration:  1200 Loss: 98.6025 Accuracy: 23.78%
Iteration:  1300 Loss: 96.3874 Accuracy: 25.79%
Iteration:  1400 Loss: 94.1802 Accuracy: 28.13%
Iteration:  1500 Loss: 91.9407 Accuracy: 30.72%
Iteration:  1600 Loss: 89.7201 Accuracy: 33.27%
Iteration:  1700 Loss: 87.6342 Accuracy: 35.36%
Iteration:  1800 Loss: 85.4570 Accuracy: 37.82%
Iteration:  1900 Loss: 83.3237 Accuracy: 40.09%
Iteration:  2000 Loss: 81.2965 Accuracy: 42.11%
Iteration:  2100 Loss: 79.210